In [1]:
import numpy as np
from tqdm.notebook import tqdm, trange
from alive_progress import alive_bar
import time # to be used in loop iterations
import caffeine
from multiprocess import Pool
import matplotlib.pyplot as plt
from queue import Queue
from threading import Thread
np.random.seed(1234)

In [3]:
#Ground truth data distribution
def random_sample_anisotropic(d, Delta, N, s_0):
    y = np.random.randint(0,2,N)
    y = np.where(y == 0, -1, y)
    x = np.zeros((N,d))
    for i in range(N):
        if y[i] == 1:
            x[i,:] = np.random.multivariate_normal([0]*d, np.diag([(1+Delta)**2]*s_0 + [1]*(d-s_0)))
        if y[i] == -1:
            x[i,:] = np.random.multivariate_normal([0]*d, np.diag([(1-Delta)**2]*s_0 + [1]*(d-s_0)))
    return x.reshape(N,d),y

## SGD dynamics

## PDE simulation

In [137]:
from scipy.stats import norm

def q_plus(r1, r2, b, Delta):
    root_plus = np.sqrt((1+Delta)**2*r1**2+r2**2)
    return b*norm.cdf(b/(root_plus))+root_plus*norm.pdf(b/root_plus)

def q_minus(r1, r2, b, Delta):
    root_minus = np.sqrt((1-Delta)**2*r1**2+r2**2)
    return b*norm.cdf(b/(root_minus))+root_minus*norm.pdf(b/root_minus)


def dq_minus(r1, r2, b, Delta):
    root_minus = np.sqrt((1-Delta)**2*r1**2+r2**2)
    return b*norm.cdf(b/(root_minus))+root_minus*norm.pdf(b/root_minus)

def dq_plus(r1, r2, b, Delta):
    from scipy.stats import norm
    d_normal = lambda x: -x*norm.pdf(x)
    
    root_plus = np.sqrt((1+Delta)**2*r1**2+r2**2)
    
    droot_plus_1 = (1+Delta)**2*r1/root_plus
    droot_plus_2 = r2/root_plus
    
    db = norm.cdf(b/(root_plus)) + b/(root_plus)*norm.pdf(b/root_plus) + d_normal(b/root_plus)
    dr1 = -droot_plus_1/(root_plus**2)*b**2*norm.pdf(b/(root_plus))\
                    +droot_plus_1*norm.pdf(b/root_plus) -b*droot_plus_1/(root_plus**2)*d_normal(b/root_plus)
    dr2 = -droot_plus_2/(root_plus**2)*b**2*norm.pdf(b/(root_plus))\
                    +droot_plus_2*norm.pdf(b/root_plus) -b*droot_plus_2/(root_plus**2)*d_normal(b/root_plus)
    return db, dr1, dr2


def dq_minus(r1, r2, b, Delta):
    from scipy.stats import norm
    d_normal = lambda x: -x*norm.pdf(x)
    
    root_minus = np.sqrt((1-Delta)**2*r1**2+r2**2)
    
    droot_minus_1 = (1-Delta)**2*r1/root_minus
    droot_minus_2 = r2/root_minus
    
    db = norm.cdf(b/(root_minus)) + b/(root_minus)*norm.pdf(b/root_minus) + d_normal(b/root_minus)
    dr1 = -droot_minus_1/(root_minus**2)*b**2*norm.pdf(b/(root_minus))\
                    +droot_minus_1*norm.pdf(b/root_minus) -b*droot_minus_1/(root_minus**2)*d_normal(b/root_minus)
    dr2 = -droot_minus_2/(root_minus**2)*b**2*norm.pdf(b/(root_minus))\
                    +droot_minus_2*norm.pdf(b/root_minus) -b*droot_minus_2/(root_minus**2)*d_normal(b/root_minus)
    return db, dr1, dr2


def R_inf(r, Delta):
    #shape of r is (J, 4)
    a = r[:,0]
    b = r[:,1]
    r1 = r[:,2]
    r2 = r[:,3]
    
    plus = a*q_plus(r1, r2, b, Delta)
    minus = a*q_minus(r1, r2, b, Delta)

    mean_q_plus = np.mean(plus)
    mean_q_minus = np.mean(minus)
    return 0.5 * (1 - mean_q_plus) ** 2 + 0.5 * (1 + mean_q_minus) ** 2

def grad_R_inf(r, Delta):
    
    #shape of r: (J, 4)
    #shape of grad: (J, 4)
    
    J = len(r)
    
    #shape of r is (J, 4)
    a = r[:,0]
    b = r[:,1]
    r1 = r[:,2]
    r2 = r[:,3]
    
    plus = a*q_plus(r1, r2, b, Delta)
    minus = a*q_minus(r1, r2, b, Delta)

    mean_q_plus = np.mean(plus)
    mean_q_minus = np.mean(minus)
    
    db_plus, dr1_plus, dr2_plus = dq_plus(r1, r2, b, Delta)
    db_minus, dr1_minus, dr2_minus = dq_minus(r1, r2, b, Delta)
    
    grad = np.zeros((J,4))
    #da
    grad[:,0] = -1/J*plus*mean_q_plus + 1/J*minus*mean_q_minus
    #db
    grad[:,1] = -1/J*db_plus*mean_q_plus + 1/J*db_minus*mean_q_minus
    #dr1
    grad[:,2] = -1/J*dr1_plus*mean_q_plus + 1/J*dr1_minus*mean_q_minus
    #dr2
    grad[:,3] = -1/J*dr2_plus*mean_q_plus + 1/J*dr2_minus*mean_q_minus
    
    return grad

def update(r, dt, grad):
    J = len(r)
    return J * grad * dt

In [148]:
#simulate pde
def pde_sim(d = 320, J=1, epsilon = 2e-4, Delta = 0.2, s_0 = 60):
    Interval = 10**np.linspace(-10,np.log10(epsilon*1e7), int(1e4))
    risk = []
    separations = [Interval[k+1]-Interval[k] for k in range(len(Interval)-1)]
    #initialise params
    Z = np.random.multivariate_normal([0]*d, np.diag([(0.8)**2/d]*d), J).reshape(J, d)
    a = np.ones(J)
    b = np.ones(J)
    r1 = np.linalg.norm(Z[:,:s_0], axis = 1)
    r2 = np.linalg.norm(Z[:,s_0:], axis = 1)
    r = np.vstack((a,b,r1,r2)).reshape(J,4)
    
    for k in trange(len(separations)):
        dt = separations[k]
        grad = grad_R_inf(r, Delta)
        r = r + (Interval[k])**(-0.25)*update(r,dt, grad)
        error = R_inf(r, Delta)
        risk.append(error)
        print(error)
    return risk

In [149]:
risk = pde_sim(d = 320, J = 400, epsilon = 2e-4, Delta = 0.6, s_0 = 60)

  0%|          | 0/9999 [00:00<?, ?it/s]

1.4997382461012876
1.4997382460569824
1.499738246012575
1.4997382459680644
1.4997382459234527
1.4997382458787378
1.499738245833921
1.499738245789001
1.4997382457439763
1.4997382456988486
1.4997382456536177
1.4997382456082824
1.4997382455628432
1.4997382455172992
1.4997382454716504
1.499738245425897
1.4997382453800379
1.4997382453340737
1.4997382452880035
1.4997382452418269
1.4997382451955452
1.4997382451491559
1.499738245102661
1.4997382450560586
1.4997382450093497
1.4997382449625325
1.4997382449156078
1.4997382448685752
1.4997382448214351
1.4997382447741854
1.4997382447268273
1.4997382446793606
1.499738244631785
1.4997382445840994
1.4997382445363048
1.4997382444883998
1.499738244440385
1.4997382443922596
1.499738244344023
1.4997382442956766
1.4997382442472176
1.499738244198648
1.4997382441499671
1.499738244101174
1.4997382440522682
1.4997382440032492
1.4997382439541185
1.499738243904875
1.499738243855518
1.4997382438060467
1.4997382437564626
1.4997382437067641
1.4997382436569517
1.499

1.4997382118204452
1.4997382116972984
1.4997382115738673
1.4997382114501532
1.4997382113261544
1.49973821120187
1.4997382110772999
1.499738210952443
1.4997382108272999
1.4997382107018684
1.4997382105761492
1.49973821045014
1.4997382103238406
1.4997382101972514
1.4997382100703713
1.4997382099431993
1.4997382098157346
1.4997382096879763
1.499738209559925
1.4997382094315777
1.4997382093029368
1.499738209174
1.4997382090447662
1.499738208915235
1.4997382087854059
1.4997382086552788
1.499738208524852
1.4997382083941249
1.4997382082630974
1.499738208131769
1.4997382080001378
1.4997382078682049
1.499738207735967
1.4997382076034265
1.4997382074705805
1.4997382073374288
1.4997382072039707
1.4997382070702066
1.4997382069361342
1.4997382068017533
1.4997382066670637
1.4997382065320646
1.4997382063967544
1.4997382062611335
1.4997382061252007
1.4997382059889544
1.4997382058523956
1.4997382057155226
1.499738205578335
1.4997382054408315
1.499738205303011
1.4997382051648744
1.4997382050264205
1.4997382

1.499738112025302
1.4997381116726378
1.499738111319164
1.4997381109648757
1.499738110609773
1.4997381102538532
1.4997381098971159
1.4997381095395574
1.499738109181177
1.499738108821972
1.4997381084619408
1.499738108101082
1.499738107739393
1.4997381073768727
1.4997381070135176
1.4997381066493276
1.4997381062842994
1.4997381059184323
1.4997381055517234
1.499738105184171
1.4997381048157739
1.499738104446529
1.4997381040764346
1.49973810370549
1.4997381033336916
1.4997381029610386
1.4997381025875276
1.4997381022131588
1.499738101837928
1.499738101461834
1.4997381010848758
1.4997381007070503
1.4997381003283565
1.4997380999487906
1.499738099568353
1.4997380991870393
1.4997380988048494
1.4997380984217803
1.4997380980378303
1.4997380976529975
1.499738097267279
1.4997380968806733
1.4997380964931788
1.4997380961047937
1.4997380957155142
1.49973809532534
1.499738094934268
1.4997380945422976
1.4997380941494252
1.4997380937556488
1.4997380933609674
1.4997380929653779
1.4997380925688786
1.499738092

1.4997378252266471
1.4997378242143826
1.4997378231997904
1.4997378221828643
1.4997378211635997
1.4997378201419906
1.499737819118033
1.4997378180917191
1.4997378170630453
1.4997378160320056
1.499737814998595
1.4997378139628075
1.4997378129246377
1.4997378118840805
1.499737810841129
1.4997378097957803
1.4997378087480269
1.4997378076978636
1.4997378066452849
1.4997378055902866
1.4997378045328613
1.4997378034730033
1.4997378024107084
1.4997378013459703
1.4997378002787833
1.4997377992091423
1.499737798137041
1.4997377970624732
1.499737795985435
1.4997377949059203
1.4997377938239218
1.4997377927394349
1.4997377916524541
1.4997377905629736
1.4997377894709871
1.4997377883764893
1.4997377872794746
1.499737786179936
1.4997377850778695
1.4997377839732682
1.4997377828661262
1.499737781756438
1.4997377806441983
1.4997377795293996
1.4997377784120376
1.4997377772921057
1.4997377761695976
1.4997377750445087
1.4997377739168314
1.4997377727865613
1.4997377716536915
1.499737770518216
1.4997377693801297
1

...

1.2023277313786496
1.2022633748749612
1.2021992444547693
1.2021353397949308
1.202071660571445
1.2020082064594615
1.2019449771332782
1.2018819722663456
1.2018191915312677
1.2017566345998043
1.2016943011428736
1.2016321908305514
1.201570303332077
1.2015086383158515
1.201447195449442
1.2013859743995823
1.201324974832175
1.201264196412294
1.2012036388041845
1.2011433016712674
1.2010831846761378
1.201023287480571
1.2009636097455207
1.2009041511311214
1.200844911296691
1.2007858899007338
1.2007270866009383
1.2006685010541829
1.2006101329165362
1.2005519818432573
1.2004940474887986
1.2004363295068097
1.2003788275501346
1.200321541270817
1.2002644703201004
1.20020761434843
1.200150973005455
1.2000945459400278
1.2000383328002096
1.1999823332332697
1.1999265468856861
1.1998709734031485
1.1998156124305612
1.1997604636120431
1.1997055265909273
1.1996508010097682
1.1995962865103376
1.1995419827336298
1.1994878893198604
1.1994340059084725
1.1993803321381322
1.1993268676467346
1.1992736120714045
1.19

1.1902568933235071
1.1902528805096209
1.1902489096334041
1.1902449803675734
1.1902410923862305
1.190237245364867
1.1902334389803668
1.1902296729110124
1.1902259468364869
1.190222260437878
1.1902186133976842
1.1902150053998148
1.1902114361295963
1.1902079052737746
1.1902044125205191
1.190200957559427
1.1901975400815237
1.1901941597792682
1.1901908163465573
1.1901875094787266
1.1901842388725534
1.1901810042262613
1.190177805239522
1.1901746416134587
1.1901715130506476
1.1901684192551212
1.1901653599323712
1.1901623347893509
1.1901593435344764
1.1901563858776307
1.1901534615301643
1.1901505702048978
1.1901477116161239
1.1901448854796102
1.1901420915126002
1.190139329433814
1.1901365989634538
1.1901338998231998
1.1901312317362172
1.1901285944271542
1.1901259876221442
1.1901234110488088
1.1901208644362562
1.1901183475150838
1.1901158600173793
1.1901134016767216
1.1901109722281815
1.1901085714083217
1.1901061989551984
1.1901038546083629
1.1901015381088595
1.190099249199228
1.1900969876235037

1.1899400462739107
1.1899400457304503
1.1899400452048257
1.1899400446964943
1.1899400442049266
1.1899400437296097
1.1899400432700453
1.1899400428257487
1.1899400423962474
1.1899400419810842
1.1899400415798147
1.1899400411920056
1.1899400408172378
1.1899400404551033
1.1899400401052058
1.189940039767161
1.1899400394405943
1.1899400391251438
1.189940038820457
1.1899400385261922
1.1899400382420167
1.189940037967609
1.1899400377026557
1.1899400374468534
1.1899400371999078
1.189940036961533
1.1899400367314514
1.1899400365093935
1.1899400362950985
1.1899400360883146
1.1899400358887942
1.1899400356963
1.1899400355106011
1.1899400353314735
1.1899400351586997
1.18994003499207
1.1899400348313793
1.1899400346764302
1.1899400345270308
1.1899400343829956
1.1899400342441449
1.1899400341103032
1.189940033981302
1.1899400338569779
1.1899400337371713
1.1899400336217296
1.189940033510504
1.189940033403349
1.189940033300126
1.1899400332007006
1.1899400331049408
1.1899400330127208
1.1899400329239171
1.1899

1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.18994003

1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.189940030754682
1.18994003

In [160]:
epsilon = 2e-4
Interval = 10**np.linspace(-10, np.log10(epsilon*1e7), int(1e4))

discretised_pde_1 = lambda k: risk[np.abs(Interval - epsilon**(4/4)*k).argmin()]
disc_pde_loss_1 = np.vectorize(discretised_pde_1)
iters = np.arange(1, int(1e7)+1)

In [ ]:
plt.plot(iters, disc_pde_loss_1(iters))
plt.xscale('log')

In [119]:
J = 400
d = 320
s_0 = 60
Delta = 0.6
Z = np.random.multivariate_normal([0]*d, np.diag([(0.8)**2/d]*d), J).reshape(J, d)
a = np.ones(J)
b = np.ones(J)
r1 = np.linalg.norm(Z[:,:s_0], axis = 1)
r2 = np.linalg.norm(Z[:,s_0:], axis = 1)
r = np.vstack((a,b,r1,r2)).reshape(J,4)
R_inf(r, Delta)

1.4952621223523412